# Track Results Overview

Per Jimmy, 

> Also one should always track the results of the experiments to be able to compare different approaches. Let's create pandas DataFrame for this purpose. 

I couldn't say it better. 

This python notebook creates a table in our instacart.db and initializes it with a dummy model. This notebook only needs to be run once to initialize the testing dataset

# Initialize Results Tracking Table

In [6]:
con = sqlite3.connect("instacart.db")
cur = con.cursor()

# Create results table
cur.execute("DROP TABLE IF EXISTS model_results;")
cur.execute("CREATE TABLE model_results (Model string, F1 float, True_Positives integer, "
            "False_Positives integer, False_Negatives integer);")

con.commit()
con.close()

# Create Dummy Model to Test

Let's predict 'None' for every single order, just to calculate F1 and append to our new results tables

In [16]:
# dictionary of orders where the products predicted will all be none
dummy_results = {}

# Load the actual results (courtesy of another notebook) to build off of:
%run Load_actual_results.ipynb

# Convert evaluation framework: results[order_id] = set([product1, product2, ...])
for row in actual_results_raw.itertuples():
    if row.order_id not in dummy_results:
        dummy_results[row.order_id] = set()
    dummy_results[row.order_id].add('None')
  


# Load f1 scoring function

In [17]:
# Read in F1 Score: 
%run F1_Score.ipynb


# Score dummy model

In [18]:
# Calculate F1 on the dummy_results
dummy_results = f1(dummy_results,actual_results)

True Positives:  2586
False Positives: 37002
False Negatives: 251536
Precision:       0.0653228250985147
Recall:          0.010176214574102203
----------------------------
F1: 0.017609206360014982


# Load to model_results

In [19]:
con = sqlite3.connect("instacart.db")
cur = con.cursor()

cur.execute("INSERT INTO model_results (Model, F1, True_Positives, "
            "False_Positives, False_Negatives) VALUES (?, ?, ?, ?, ?);", 
            list(("Dummy Model",) + dummy_results) )

con.commit()
con.close()

# Print contents of table

In [22]:
import pandas as pd

con = sqlite3.connect("instacart.db")
cur = con.cursor()


print(pd.read_sql_query("SELECT * FROM model_Results;", con))

con.close()

         Model        F1  True_Positives  False_Positives  False_Negatives
0  Dummy Model  0.017609            2586            37002           251536
